In [1]:
from google_images_download import google_images_download

In [3]:
import os

In [4]:
os.getcwd()

'C:\\Users\\An_asus\\Desktop\\Programming\\JupyterNotebook\\tensorflow'

In [5]:
data_path = 'C:/Users/An_asus/Desktop/AI/data sets/google_img_dwn/FER/'

In [11]:
driver_path = 'C:/Users/An_asus/Desktop/chrome driver/chromedriver.exe'

In [12]:
args = {'keywords': 'happy human face',
        'output_directory': data_path,
        'image_directory': 'happy', 
        'silent_mode': True,    
        'limit': 1000,
        'chromedriver': driver_path}

In [13]:
response = google_images_download.googleimagesdownload()
response.download(args)

Getting you a lot of images. This may take a few moments...
Reached end of Page.


Unfortunately all 1000 could not be downloaded because some images were not downloadable. 0 is all we got for this search filter!


({'happy human face': []}, 0)

In [14]:
from facenet_pytorch import MTCNN
from PIL import Image

img_path = 'C:/Users/An_asus/Desktop/sad man.jpg'
img = Image.open(img_path)

model = MTCNN(keep_all=True)
bboxes, preds = model.detect(img)

C:\Users\An_asus\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [15]:
bboxes

array([[ 544.0206,  439.0514, 1207.0353, 1318.7609]], dtype=float32)

In [16]:
preds

array([0.9998766], dtype=float32)

In [17]:

import torch.nn as nn
from torchvision.models import resnet50

def init_grayscale_resnet(init_weights=None):
    """Initialize a pretrained Resnet-50 model and change the first layer to be a one-channel 2D convolution
    
    Parameters
    ----------
    init_weights: nn.Tensor, optional
        A tensor containing the initial weights for the new convolution layer with shape (64, 1, 7, 7).
        If left as None, the parameters will be initialized using the kaiming uniform distribution.
    
    Returns
    -------
    gray_model: nn.Module
    """
    gray_model = resnet50(pretrained=True)
    if init_weights is None:
        # change the first layer to use 1x7x7-sized kernels instead of 3x7x7-sized kernels
        w = torch.zeros((64, 1, 7, 7))
        nn.init.kaiming_uniform_(w, a=math.sqrt(5))
    else:
        w = init_weights
    gray_model.conv1.weight.data = w
    return gray_model

In [18]:
from torchvision.models import resnet50

# The layers of Resnet-50 are organized into blocks
# The last three blocks contain 11 trainable layers in total (10, 0, and 1, in that order)
# We will go ahead and freeze the parameters of all but the last 11 layers in the model
model = resnet50()
for model_block in list(model.children())[:-3]:
    for param in model_block.parameters():
        param.requires_grad = False